# Minimal Example
1. specify models to negotiate
2. run negotiation
3. process and results
4. example for commandline usage

In [ ]:
import yaml
from src.agents import NegotiationAgent
from src.protocols import NegotiationProtocol
from src.models import OpenAIModel
from src.games import Game, Issue

# where to save results
save_dir='my_example_logs/'

# which model and model provider should be used?
agent_1_model_name = 'gpt-3.5-turbo'
agent_1_model_provider = 'azure'

agent_2_model_name = 'gpt-3.5-turbo'
agent_2_model_provider = 'azure'

offerx_model_name = 'gpt-3.5-turbo'
offerx_model_provider = 'azure'

# which agents are used?
# agent_1_desc = yaml.safe_load(open('data/agents/anon.yaml'))
agent_1_desc = {
    'internal_description': {
        'name': 'You'
    },
    'external_description': {
        'name': 'Representative'
    }
}
# agent_2_desc = yaml.safe_load(open('data/agents/anon.yaml'))
agent_2_desc = {
    'internal_description': {
        'name': 'You'
    },
    'external_description': {
        'name': 'Representative'
    }
}

# which game setting should be played? (see data/games for examples)
game = yaml.safe_load(open("data/games/generic-rental-agreement.yaml"))
# which issues should be negotiated over? (see data/issues for examples)
issues = ["gen-ra-rent.yaml"]
# how should the different issues be weighted in importance? (only relevant for >1 issues)
issue_weights = [[1], [1]]


# instantiate game
game = Game(issues=issues, issue_weights=issue_weights, **game)
# instantiate agents
agent_1 = NegotiationAgent(model=OpenAIModel(model_name=agent_1_model_name, model_provider=agent_1_model_provider),
                           model_name=agent_1_model_name, model_provider=agent_1_model_provider,
                          **agent_1_desc)
agent_2 = NegotiationAgent(model=OpenAIModel(model_name=agent_2_model_name, model_provider=agent_2_model_provider),
                           model_name=agent_2_model_name, model_provider=agent_2_model_provider,
                          **agent_2_desc
                          )
# instantiate protocol
negotiation_protocol = NegotiationProtocol(game=game,
                                           agent_1=agent_1,
                                           agent_2=agent_2,
                                           save_folder=save_dir
                                          )
# run negotiation
negotiation_protocol.run()

## Process raw transcript

In [ ]:
from src.process_transcripts import ProcessTranscript

# process raw transcript file --> saved to same folder with file name 'processed_transcripts.csv'
neg_eval = ProcessTranscript(save_dir=save_dir, game=game, update=False)
neg_eval.compute_metrics()

In [ ]:
import pandas as pd

# inspect results
df = pd.read_csv(f'{save_dir}/processed_negotiation.csv')
df.head()

## Run from command line

In [ ]:
# which model and model provider should be used?
agent_1_model_name = 'gpt-3.5-turbo'
agent_1_model_provider = 'azure'

agent_2_model_name = 'gpt-3.5-turbo'
agent_2_model_provider = 'azure'

offerx_model_name = 'gpt-3.5-turbo'
offerx_model_provider = 'azure'

agent_1_desc = 'data/agents/anon.yaml'
agent_2_desc = 'data/agents/anon.yaml'


# which game setting should be played? (see data/games for examples)
game = "data/games/generic-rental-agreement.yaml"
# which issues should be negotiated over? (see data/issues for examples)
issues = '["gen-ra-rent.yaml"]'
# how should the different issues be weighted in importance? (only relevant for >1 issues)
issue_weights = '[[1], [1]]'
# what is the maximum, aggregate utility that can be obtained across issues?
scale = '[[100], [100]]'

# putting everything together -> note the extra single quotes around 'array' type values!
run_str = f"""
python src/run_scratch.py \
++experiments.agent_1.model_name={agent_1_model_name} \
++experiments.agent_1.model_provider={agent_1_model_provider} \
++experiments.agent_2.model_name={agent_2_model_name} \
++experiments.agent_2.model_provider={agent_2_model_provider} \
++offer_extraction_model_name={offerx_model_name} \
++offer_extraction_model_provider={offerx_model_provider} \
++experiments.agent_1.agent_desc={agent_1_desc} \
++experiments.agent_2.agent_desc={agent_2_desc} \
++experiments.game.name={game} \
'++experiments.game.issues={issues}' \
'++experiments.game.issue_weights={issue_weights}' \
'++experiments.game.scale={scale}' \
++max_rounds=1"""

In [ ]:
!$run_str